In [14]:
import pandas as pd
import numpy as np
import math
import random

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import f1_score, make_scorer, r2_score, accuracy_score
from sklearn.model_selection import GridSearchCV, ShuffleSplit





In [6]:
test_data = pd.read_csv("test.csv")
train_data = pd.read_csv("train.csv")
test_survive = pd.read_csv('gender_submission.csv')
test_data['Survived'] = test_survive['Survived']
test_data.head(3)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0


In [7]:
original_data = train_data.copy()
full_data = [train_data, test_data]
train_data['cabin'] = train_data['Cabin'].apply(lambda x:0 if type(x) == float else 1)
test_data['cabin'] = test_data['Cabin'].apply(lambda x:0 if type(x) == float else 1)
train_data['Sex'] = train_data['Sex'].map({'female':0, 'male':1}).astype(int)
test_data['Sex'] = test_data['Sex'].map({'female':0, 'male':1}).astype(int)

In [8]:


x_train = []
y_train = []
x_test = []
y_test = []


def cleaning(data):
    x = []
    y = []
    for i in range(len(data)):
        if math.isnan(data['Age'][i]):
            continue
        if data['Age'][i] >=0 and data['Age'][i] <= 10:
            age = 0
        elif data['Age'][i] > 10 and data['Age'][i] <= 20:
            age = 1
        elif data['Age'][i] > 20 and data['Age'][i] <= 50:
            age = 2
        else:
            age = 3
            
        if data['Embarked'][i] == 'S':
            embarked = 0
        elif data['Embarked'][i] == 'C':
            embarked = 1
        elif data['Embarked'][i] == 'Q':
            embarked = 2
        else:
            embarked = 3
        x.append([data['Sex'][i],age, data['SibSp'][i], data['Parch'][i],embarked, data['Pclass'][i], data['cabin'][i]])
        y.append (data['Survived'][i])
    return x, y
x_train, y_train = cleaning(train_data)
x_test, y_test = cleaning(test_data)



In [9]:
count = 0
for i in x_train:
    print(i)
    if(count > 10):
        break
    count += 1

count = 0
print("\n")
for i in x_test:
    print(i)
    if(count > 10):
        break
    count += 1


[1, 2, 1, 0, 0, 3, 0]
[0, 2, 1, 0, 1, 1, 1]
[0, 2, 0, 0, 0, 3, 0]
[0, 2, 1, 0, 0, 1, 1]
[1, 2, 0, 0, 0, 3, 0]
[1, 3, 0, 0, 0, 1, 1]
[1, 0, 3, 1, 0, 3, 0]
[0, 2, 0, 2, 0, 3, 0]
[0, 1, 1, 0, 1, 2, 0]
[0, 0, 1, 1, 0, 3, 1]
[0, 3, 0, 0, 0, 1, 1]
[1, 1, 0, 0, 0, 3, 0]


[1, 2, 0, 0, 2, 3, 0]
[0, 2, 1, 0, 0, 3, 0]
[1, 3, 0, 0, 2, 2, 0]
[1, 2, 0, 0, 0, 3, 0]
[0, 2, 1, 1, 0, 3, 0]
[1, 1, 0, 0, 0, 3, 0]
[0, 2, 0, 0, 2, 3, 0]
[1, 2, 1, 1, 0, 2, 0]
[0, 1, 0, 0, 1, 3, 0]
[1, 2, 2, 0, 0, 3, 0]
[1, 2, 0, 0, 0, 1, 0]
[0, 2, 1, 0, 0, 1, 1]


In [10]:
random.seed(42)

In [28]:
model = DecisionTreeClassifier(random_state=42)

cv_sets = ShuffleSplit(n_splits = 10, test_size = 0.20, random_state = 0)



scorer = make_scorer(accuracy_score)

parameters = {'max_depth':[5,6,7], 'min_samples_leaf':[3,4,5], 'min_samples_split':[3,4,5]}

grid_model = GridSearchCV(estimator=model, param_grid=parameters, scoring=scorer, cv=cv_sets)
model_fit = grid_model.fit(x_train, y_train)
best_model = model_fit.best_estimator_
print(best_model)

best_model.fit(x_train, y_train)
#model.fit(x_train, y_train)
train_prediction = best_model.predict(x_train)
test_prediction = best_model.predict(x_test)

train_score = accuracy_score(train_prediction, y_train)
test_score = accuracy_score(test_prediction, y_test)
print(f"train_score = {train_score}")
print(f"test_score = {test_score}")


DecisionTreeClassifier(max_depth=5, min_samples_leaf=3, min_samples_split=3,
                       random_state=42)
train_score = 0.834733893557423
test_score = 0.8554216867469879
